<a href="https://colab.research.google.com/github/partizanos/advanced_image_processing/blob/master/tp4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tp4 image processing JPEG -- DIMITRIS PROIOS

## Exercise 1 (a) Implement by yourself the JPEG compression algorithm described above.

In [0]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import requests
from PIL import Image
from io import BytesIO

response = requests.get("https://raw.githubusercontent.com/partizanos/advanced_image_processing/master/TP/TP4/test.png")
im = Image.open(BytesIO(response.content))
img = np.array(im)
crop = lambda img:img[:max_M, :max_N, :]
img = crop(img )

1. RGB color space to YCbCr color space conversion:

In [0]:
def extract_colors(img):
  R_DIMENSION = img[:, :, 0]
  G_DIMENSION = img[:, :, 1]
  B_DIMENSION = img[:, :, 2]
  return R_DIMENSION, G_DIMENSION, B_DIMENSION


def YCbCr(R_DIMENSION,G_DIMENSION,B_DIMENSION): 
  Y = np.add(np.add(0.299 * R_DIMENSION, 0.587 * G_DIMENSION), 0.114 * B_DIMENSION)
  Cb = - 0.1687 * R_DIMENSION - 0.3313 * G_DIMENSION + 0.5 * B_DIMENSION + 128
  Cr = 0.5 * R_DIMENSION - 0.4187 * G_DIMENSION - 0.0813 * B_DIMENSION + 128
  R_DIMENSION = Y + 1.402 * (Cr - 128)
  G_DIMENSION = Y - 0.34414 * (Cb - 128) - 0.71414 * (Cr - 128)
  B_DIMENSION = Y + 1.772 * (Cb - 128)
  return Y, Cb, Cr, R_DIMENSION, G_DIMENSION, B_DIMENSION

Y, Cb, Cr, R_DIMENSION, G_DIMENSION, B_DIMENSION = YCbCr(*extract_colors(img))

2. Keep the Y component.
Downsample the Cb and Cr components in 2 times.
Apply DCT compression to each component.

In [113]:
def downsample_image(img, skip):
    return img[::skip,::skip]

d_cb  = downsample_image(Cb, 2)
d_cb  = downsample_image(d_cb, 2)
d_cr  = downsample_image(Cr, 2)
d_cr  = downsample_image(d_cr, 2)
Y[0].shape

(1296,)

3. Preprocessing for DCT transformation

3.1 Split the image into 8 × 8 non-overlapping blocks

In [114]:
import math
M, N, _ = img.shape
max_M = int(math.modf(M/8)[1] *8)
max_N = int(math.modf(N/8)[1] *8)
global_mean = 128
number_octads = int(max_M* max_N / 64)
number_octads, Y.shape, max_M, max_N

(17496, (864, 1296), 864, 1296)

3.2 In each block subtract global mean computed as 2 k−1 , where k is the number of gray
levels in the image

In [0]:
Y_r = Y.reshape((number_octads, 8,8)) - global_mean
Cb_r = Cb.reshape((number_octads, 8,8)) - global_mean
Cr_r = Cr.reshape((number_octads, 8,8)) - global_mean

Step 4. DCT transformation per block: T (u, v)

In [116]:
from scipy import fftpack 

Y_dct = fftpack.dct(Y_r)
Cb_dct = fftpack.dct(Cb_r)
Cr_dct = fftpack.dct(Cr_r)

((17496, 8, 8), array([-27.7724    ,   6.61062163,   6.95819743,   1.63500718,
          2.82842712,  -0.57715909,  -0.5674042 ,  -2.80283902]))

Step 5. Block coefficients quantization:

$T(u,v) = round \frac{T(u,v)} {Z(u,v)}  $

In [1]:
Z = [
    [16, 11, 10, 16, 24, 40, 51, 61],
    [12, 12, 14, 19, 26, 28, 60, 55],
    [14, 13, 16, 24, 40, 57, 69, 56],
    [14, 17, 22, 29, 51, 87, 80, 62],
    [18, 22, 37, 56, 68, 109, 103, 77],
    [24, 35, 55, 64, 81, 104, 113, 92],
    [49, 64, 78, 87, 103, 121, 120, 101],
    [72, 92, 95, 98, 112, 100, 103, 99]
]

T_Y = np.round(Y_dct/Z)
T_cb = np.round(Cb_dct/Z)
T_cr = np.round(Cr_dct/Z)

T_cr.shape, T_cr[0, :]

NameError: ignored

Step 6. Symbols encoding:
6.1  Zig-zag scanning


In [0]:
# https://rosettacode.org/wiki/Zig-zag_matrix#Python
import operator

def zigzag(n):
    '''zigzag rows'''
    def compare(xy):
        x, y = xy
        return (x + y, -y if (x + y) % 2 else y)
      
    xs = range(n)
    
    return {index: n for n, index in enumerate(sorted(
        ((x, y) for x in xs for y in xs),
        key=compare
    ))}


x = zigzag(8)
sorted_x = sorted(x.items(), key=operator.itemgetter(1))



# [ for i in sorted_x for arr in []]

In [2]:
Y_b = T_Y.reshape((max_M, max_N))
Y_b.shape

NameError: ignored

6.2 Huffman coding

In [0]:
#https://gist.github.com/mreid/fdf6353ec39d050e972b
# Example Huffman coding implementation
# Distributions are represented as dictionaries of { 'symbol': probability }
# Codes are dictionaries too: { 'symbol': 'codeword' }

def huffman(p):
    '''Return a Huffman code for an ensemble with distribution p.'''
    assert(sum(p.values()) == 1.0) # Ensure probabilities sum to 1

    # Base case of only two symbols, assign 0 or 1 arbitrarily
    if(len(p) == 2):
        return dict(zip(p.keys(), ['0', '1']))

    # Create a new distribution by merging lowest prob. pair
    p_prime = p.copy()
    a1, a2 = lowest_prob_pair(p)
    p1, p2 = p_prime.pop(a1), p_prime.pop(a2)
    p_prime[a1 + a2] = p1 + p2

    # Recurse and construct code on new distribution
    c = huffman(p_prime)
    ca1a2 = c.pop(a1 + a2)
    c[a1], c[a2] = ca1a2 + '0', ca1a2 + '1'

    return c

def lowest_prob_pair(p):
    '''Return pair of symbols from distribution p with lowest probabilities.'''
    assert(len(p) >= 2) # Ensure there are at least 2 symbols in the dist.

    sorted_p = sorted(p.items())
    return sorted_p[0][0], sorted_p[1][0]

# Example execution
# ex1 = { 'a': 0.5, 'b': 0.25, 'c': 0.25 }
# huffman(ex1) # => {'a': '0', 'c': '10', 'b': '11'}


In [0]:
# merge all blocks 


In [121]:
T_Y.shape

(17496, 8, 8)